In [5]:
import random
import string
# special_chars = r".+*?^$()[]{}\|"

def randstr(N=3):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(N))

SPECIAL_CHARS = r'\[]'

import regex as re
from copy import deepcopy
EXIT_STATE = 'end'
START_STATE = 'start'

def make_or_nfa(pattern):
    assert pattern[0] != '('
    pattern = list(pattern)
    
    groups = []
    current_group = ''
    while pattern:
        next_char = pattern.pop(0)

        if next_char == '\\':
            assert pattern != '', 'group can\'t end in odd # backslashes'
            current_group += next_char+pattern.pop(0)
        elif next_char == '|':
            groups.append(current_group)
            current_group = ''
        else:
            current_group += next_char
    
    groups.append(current_group)

    groups = [make_nfa(group) for group in groups]
    group_starts = []
    nfa = {}
    for group_nfa in groups:
        new_start, new_end = rename_boundary_states(group_nfa)
        group_nfa[new_end] = {'': {EXIT_STATE}}
        group_starts.append(new_start)
        nfa.update(group_nfa)
    nfa[START_STATE] = {'': set(group_starts)}
    return nfa
        


open_to_close = {
    '[': ']',
    '(': ')',
    '\{': '\}'
}

def extract_bracketed_group(s):
    # s == a string that starts with a valid opening character: [, ), {
    open_ = s[0]
    close_ = open_to_close[open_]
    s = s[1:]
    counter = 1
    skip = False
    for i, c in enumerate(s):
        if skip == True:
            skip = False
            continue
        if c == '\\':
            skip = True
            continue
        if c == open_:
            counter += 1
        elif c == close_:
            counter -= 1
        if counter == 0:
            return open_+s[:i]+close_
    return False
        
            
extract_bracketed_group('(teb\)k[as[ti]n)]g]')

def make_char_nfa(bracket_insides: str):
    pat = '|'.join(list(bracket_insides))
    or_nfa = make_or_nfa(pat)
    return or_nfa



def combine_nfas(nfa1, nfa2):
    in_between_state = 'transition_'+randstr()
    nfa1, nfa2 = deepcopy(nfa1), deepcopy(nfa2)
    for current_state, transition_rules in nfa1.items():
        for k, v in transition_rules.items():
            if EXIT_STATE in v:
                v.remove(EXIT_STATE)
                v.append(in_between_state)
    for current_state, transition_rules in nfa2.items():
        if current_state == START_STATE:
            nfa2[in_between_state] = deepcopy(transition_rules)
        break
    del nfa2[START_STATE]

    nfa1.update(nfa2)
    return nfa1

# def join_group(partial_nfa, group_nfa, current_state):

    
def rename_boundary_states(nfa):
    assert EXIT_STATE not in nfa
    new_start, new_end = randstr(), randstr()
    
    assert START_STATE in nfa
    nfa[new_start] = deepcopy(nfa[START_STATE])

    if START_STATE in nfa:
        del nfa[START_STATE]

    for state, transitions in nfa.items():
        for key, dests in transitions.items():
            assert isinstance(dests, set), dests
            if EXIT_STATE in dests:
                nfa[state][key] = (dests - {EXIT_STATE}).union({new_end})
    return new_start, new_end

    
def make_nfa(pattern):
    # returns nfa: { current_state: {next_char: next_state, ...}, ...}

    nfa = {}
    current_state = START_STATE
    pattern = list(pattern)
    while pattern:
        if pattern[0] in r'[({':
            group = extract_bracketed_group(''.join(pattern))
                
            if group == False:
                assert False, f"Unbalanced group from here on: {''.join(pattern)}"
            elif group[0] == '(':
                group_nfa = make_nfa(group[1:-1])
            elif group[0] == '[':
                group_nfa = make_char_nfa(group[1:-1])
            elif group[0] == '\{':
                assert False, r'{...} unimplemented'
            
            gp_start, gp_end = rename_boundary_states(group_nfa)
            nfa[current_state] = {
                '': {gp_start}
            }
            current_state = gp_end

            pattern = pattern[len(group):]
            continue
        else:
            c = pattern.pop(0)
            assert c not in SPECIAL_CHARS
            next_state = randstr()
            nfa[current_state] = {
                    c: {next_state}
                }
            current_state = next_state


    nfa[current_state] = {'': {EXIT_STATE}}
    return nfa

def check(nfa, s):
    matches = []
    pennies = [{'state': START_STATE, 'i': 0, 'eps_seen_states': set()}]
    while pennies:
        penny = pennies.pop()
        state = penny['state']

        next_char = s[penny['i']] if penny['i'] < len(s) else False


        for next_state in nfa.get(state, {}).get('', []):
            if next_state not in penny['eps_seen_states']:
                new_penny = deepcopy(penny)
                new_penny['eps_seen_states'].add(penny['state'])
                new_penny['state'] = next_state
                pennies.append(new_penny)
    
        if next_char in nfa.get(state, {}):
            for new_state in nfa[state][next_char]:
                new_penny = deepcopy(penny)
                new_penny['eps_seen_states'] = set()
                new_penny['state'] = new_state
                new_penny['i'] += 1
                pennies.append(new_penny)
        
        if next_char == False and penny['state'] == EXIT_STATE:
                return True
    else:
        return False
    
    
                

# nfa1 = make_nfa('ab')
nfa2 = make_nfa('[d]')
print(nfa2)


# check(nfa2, 'd')


{'start': {'': {'H1E'}}, 'UAN': {'': {'end'}}}


In [49]:
make_nfa('[df]')

or_pattern d|f


{'start': {'d': {'34G'}}, '34G': {'f': {'2Y3'}}, '2Y3': {'': ['end']}}

<regex.Match object; span=(4, 5), match='|'>


In [32]:
A

<regex.Match object; span=(4, 5), match='|'>

In [ ]:
def get_valid_empty_transition_children(nfa, penny):
    pennies = [penny]
    while True:
        for penny in pennies:
            if 

In [ ]:
def simplify_arrows(nfa):
    simplified_nfa = deepcopy(nfa)
    for state, transitions in nfa.items():
        if '' in transitions:
            new_state_set = []
            null_transitions = transitions
            seen = set(transitions)
            assert state not in transitions
            seen.add(state)
            while True:
                new = [new_state for new_state in transitions[''] if new_state not in seen]

            

In [69]:
A = ['test', 'ing']
A.remove('ing')
A

['test']

'(teb\\)k[as[ti]n)'

In [45]:
r'(?<!\)(\\)*+|'

'(?<!\\)(\\\\)*+|'